In [56]:
import requests
import bs4

In [57]:
url = 'https://kozigallas.gov.hu/publicsearch.aspx'

In [58]:
keywords = ['gyógypedagógus','tanár','tanító','óvodapedagógus','orvos', 'védőnő']

The response is an HTML page which we'll have to parse. So we'll use BeautifulSoup to take the response string and parse the HTML:

Let's also add the date and time of retrieval. We'll put the date along with the count into an array so we can easily add it as a row to a CSV.

**NOTE** We are using timezone to set UTC as the time on everything, because that is what Github Actions uses.

In [59]:
from datetime import datetime, timezone

current_date = datetime.now(timezone.utc).strftime("%m/%d/%Y")
current_time = datetime.now(timezone.utc).strftime("%H:%M:%S")

HEADERS = ['date']
FILENAME = 'vacancies_count.csv'

for keyword in keywords:
    HEADERS.append(keyword)
    print(HEADERS)

index = 2
row = [current_date]

for keyword in keywords:
    response = send_request(keyword)
    html = bs4.BeautifulSoup(response, 'html.parser')
    count = html.find(id='ctl00_ContentPlaceHolder1_JobSearchForm1_lblCount').text
    row.append(count)

import os
import csv

if not os.path.isfile(FILENAME):
   with open(FILENAME, 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(HEADERS)
    writer.writerow(row)
else: # else it exists so append without writing the header
   with open(FILENAME, 'a', newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(row)

['date', 'gyógypedagógus']
['date', 'gyógypedagógus', 'tanár']
['date', 'gyógypedagógus', 'tanár', 'tanító']
['date', 'gyógypedagógus', 'tanár', 'tanító', 'óvodapedagógus']
['date', 'gyógypedagógus', 'tanár', 'tanító', 'óvodapedagógus', 'orvos']
['date', 'gyógypedagógus', 'tanár', 'tanító', 'óvodapedagógus', 'orvos', 'védőnő']


Done! 👏